In [1]:
import numpy as np
import pandas as pd 
import glob
import time
import os 
import matplotlib.pyplot as plt 
import keras
import cv2

2025-01-21 18:35:47.582557: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("XSS_dataset.csv",encoding = 'utf-8-sig')
df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [3]:
df = df[df.columns[-2:]]
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [4]:
sentences = df['Sentence'].values
sentences[1]

'<tt onmouseover="alert(1)">test</tt>'

In [5]:
print(len(sentences))

13686


In [6]:
# Convert to ASCII

# Some charecter have values very big e.d. 8221 a dn some chinese letters 
# i am removing letters having values greater than 8222 and for rest greater 
# than 128 and smaller than 8222 assigning them values so they can be easily be normalized

def convert_to_ascii(sentence):
    sentence_ascii=[]
    for i in sentence:
        if(ord(i)<8222):
            if(ord(i)==8217):
                sentence_ascii.append(134)
            if(ord(i)==8221):
                sentence_ascii.append(129)
            if(ord(i)==8220):
                sentence_ascii.append(130)
            if(ord(i)==8216):
                sentence_ascii.append(131)
            if(ord(i)==8217):
                sentence_ascii.append(132)
            if(ord(i)==8211):
                sentence_ascii.append(133)
# if values less than 128 store them else discard them
            if(ord(i)<=128):
                sentence_ascii.append(ord(i))
            else:
                pass
    zer=np.zeros((10000))
    
    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]
    zer.shape=(100,100)
    
    return zer

In [7]:
gg=convert_to_ascii(sentences[1])
print(gg)

[[ 60. 116. 116. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [8]:
# send each sentence to be convert to ASCII

arr=np.zeros((len(sentences),100,100))
for i in range(len(sentences)):
    image=convert_to_ascii(sentences[i])
    x=np.asarray(image,dtype='float')
    image = cv2.resize(x,dsize=(100,100),interpolation=cv2.INTER_CUBIC)
    image/=128
    arr[i]=image

In [9]:
print("Input data shape: ",arr.shape)

Input data shape:  (13686, 100, 100)


In [10]:
# Reshape data for input to CNN
data = arr.reshape(arr.shape[0],100,100,1)

In [11]:
data.shape

(13686, 100, 100, 1)

In [12]:
y=df['Label'].values

In [13]:
# Split into train/test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.2, random_state = 42)


In [14]:
# import libreries for Making Model
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Activation , Conv2D, MaxPooling2D, Flatten, Dropout,MaxPool2D,BatchNormalization        

In [15]:
from sklearn.metrics import accuracy_score


# Define the fitness function
def fitness_function(params):
    # Extract hyperparameters from params
    conv1_filters, conv2_filters, conv3_filters = int(params[0]), int(params[1]), int(params[2])
    dense1_neurons, dense2_neurons = int(params[3]), int(params[4])
    learning_rate, epochs = params[5], int(params[6])

    # Build the CNN model
    model = Sequential([
        Conv2D(conv1_filters, (3, 3), activation='relu', input_shape=(100, 100, 1)),
        MaxPooling2D(2, 2),
        Conv2D(conv2_filters, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(conv3_filters, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(dense1_neurons, activation='relu'),
        Dense(dense2_neurons, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)
    
    return -acc  # Negate because GWO minimizes the fitness function

# Grey Wolf Optimizer Implementation
class GreyWolfOptimizer:
    def __init__(self, fitness_func, dim, lb, ub, n_agents=10, max_iter=20):
        self.fitness_func = fitness_func
        self.dim = dim
        self.lb = np.array(lb)
        self.ub = np.array(ub)
        self.n_agents = n_agents
        self.max_iter = max_iter

        # Initialize the positions of the wolves
        self.positions = np.random.uniform(lb, ub, (n_agents, dim))
        self.alpha_score = float('inf')
        self.beta_score = float('inf')
        self.delta_score = float('inf')
        self.alpha_pos = np.zeros(dim)
        self.beta_pos = np.zeros(dim)
        self.delta_pos = np.zeros(dim)

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.n_agents):
                fitness = self.fitness_func(self.positions[i])
                if fitness < self.alpha_score:
                    self.alpha_score, self.alpha_pos = fitness, self.positions[i].copy()
                elif fitness < self.beta_score:
                    self.beta_score, self.beta_pos = fitness, self.positions[i].copy()
                elif fitness < self.delta_score:
                    self.delta_score, self.delta_pos = fitness, self.positions[i].copy()

            # Update the positions of the wolves
            a = 2 - t * (2 / self.max_iter)
            for i in range(self.n_agents):
                for d in range(self.dim):
                    r1, r2 = np.random.rand(), np.random.rand()
                    A1, C1 = 2 * a * r1 - a, 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[d] - self.positions[i, d])
                    X1 = self.alpha_pos[d] - A1 * D_alpha

                    r1, r2 = np.random.rand(), np.random.rand()
                    A2, C2 = 2 * a * r1 - a, 2 * r2
                    D_beta = abs(C2 * self.beta_pos[d] - self.positions[i, d])
                    X2 = self.beta_pos[d] - A2 * D_beta

                    r1, r2 = np.random.rand(), np.random.rand()
                    A3, C3 = 2 * a * r1 - a, 2 * r2
                    D_delta = abs(C3 * self.delta_pos[d] - self.positions[i, d])
                    X3 = self.delta_pos[d] - A3 * D_delta

                    self.positions[i, d] = np.clip((X1 + X2 + X3) / 3, self.lb[d], self.ub[d])

        return self.alpha_pos, -self.alpha_score

# Define hyperparameter bounds
bounds = [
    [32, 128],  # Conv1 filters
    [64, 256],  # Conv2 filters
    [128, 512], # Conv3 filters
    [64, 256],  # Dense1 neurons
    [32, 128],  # Dense2 neurons
    [0.0001, 0.01],  # Learning rate
    [5, 50]     # Epochs
]

lb = [b[0] for b in bounds]
ub = [b[1] for b in bounds]

# Run GWO
gwo = GreyWolfOptimizer(fitness_function, dim=len(bounds), lb=lb, ub=ub, n_agents=5, max_iter=10)
best_params, best_score = gwo.optimize()

print("Best Parameters:", best_params)
print("Best Fitness (Accuracy):", -best_score)


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 94)     │           940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 94)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 96)     │        81,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 21, 149)    │       128,885 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 149)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14900)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 141)            │     2,101,041 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         8,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            61 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,320,759 (8.85 MB)

 Trainable params: 2,320,759 (8.85 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_147 (Conv2D)             │ (None, 98, 98, 37)     │           370 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_147               │ (None, 49, 49, 37)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_148 (Conv2D)             │ (None, 47, 47, 222)    │        74,148 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_148               │ (None, 23, 23, 222)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_149 (Conv2D)             │ (None, 21, 21, 358)    │       715,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_149               │ (None, 10, 10, 358)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_49 (Flatten)            │ (None, 35800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_147 (Dense)               │ (None, 133)            │     4,761,533 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_148 (Dense)               │ (None, 65)             │         8,710 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_149 (Dense)               │ (None, 1)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,560,469 (21.21 MB)

 Trainable params: 5,560,469 (21.21 MB)

 Non-trainable params: 0 (0.00 B)

86/86 ━━━━━━━━━━━━━━━━━━━━ 36s 417ms/step
Best Parameters: [3.20000000e+01 2.56000000e+02 3.42958224e+02 1.77901307e+02
 8.26295078e+01 1.00000000e-04 4.92971894e+01]
Best Fitness (Accuracy): -0.9952520087655223


In [33]:
# Define the best parameters
best_para = [
    3.20000000e+01,  # Conv1 Filters
    2.56000000e+02,  # Conv2 Filters
    3.42958224e+02,  # Conv3 Filters (approx.)
    1.77901307e+02,  # Dense1 Neurons (approx.)
    8.26295078e+01,  # Dense2 Neurons (approx.)
    1.00000000e-04,  # Learning Rate
    4.92971894e+01   # Epochs (approx.)
]

# Define parameter names
param_names = [
    "Conv1 Filters",
    "Conv2 Filters",
    "Conv3 Filters",
    "Dense1 Neurons",
    "Dense2 Neurons",
    "Learning Rate",
    "Epochs"
]

# Convert values to fixed-point notation
fixed_params = {
    name: (f"{val:.4f}" if val < 1 else int(round(val))) for name, val in zip(param_names, best_para)
}

# Print results in table format
print("Hyperparameter\tOptimal Value")
print("-" * 35)
for key, value in fixed_params.items():
    print(f"{key}\t{value}")


Hyperparameter	Optimal Value
-----------------------------------
Conv1 Filters	32
Conv2 Filters	256
Conv3 Filters	343
Dense1 Neurons	178
Dense2 Neurons	83
Learning Rate	0.0001
Epochs	49


In [37]:
# Print accuracy
accu = 0.9952520087655223
print(f"Best Accuracy: {accu * 100:.2f}%")

Best Accuracy: 99.53%
